In [1]:

import librosa
import soundfile
import os, glob
import numpy as np


# Modules to load data and extract features

In [2]:
# Extract features : mfcc, chroma, mel

def extract_feature(file, mfcc, chroma, mel):
    with soundfile.SoundFile(file) as file:
        X = file.read(dtype="float32")
        sample_rate=file.samplerate
        
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        
        return result

In [3]:
# Emotions in the RAVDESS dataset

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# Emotions to observe
observed_emotions = ['neutral','happy','angry','surprised']

In [4]:

# Load the data and extract features for each sound file

# wav_files = 'C:\\Users\\admin\\Downloads\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav'
wav_files = 'C:\\Users\\admin\\Downloads\\bdad-project\\sample-ravdess\\Actor_*\\*.wav'

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(wav_files):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


# Creating and training the ML model

In [13]:
# creating the model

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = load_data(test_size=0.25)


model= MLPClassifier(
            alpha=0.01, 
            batch_size=10, 
            epsilon=1e-08, 
            hidden_layer_sizes=(300,), 
            learning_rate='adaptive', 
            max_iter=500 )

model.fit(x_train, y_train)
model.score(x_test, y_test)


0.5714285714285714

In [14]:
model.score(x_train, y_train)

0.7619047619047619

# Serializing the model

In [9]:
# serialising the model object so we dont have to retrain it every time

import pickle

pickle_model = pickle.dumps(model)
# pickle.dump(model, model_pickle.pkl)


In [11]:
import pickle

model_unpickled = pickle.loads(pickle_model)

model_unpickled.score(x_test, y_test)


0.3142857142857143

In [9]:
# recording sound and storing as a wave file
# REDUNDANT MODULE: voice recording functionality has been implemented in the frontend

# import sounddevice 
# from scipy.io.wavfile import write

# fs = 44100
# seconds = 5
# print("recording...")
# record_voice = sounddevice.rec(int(seconds*fs), samplerate=fs, channels=2)
# sounddevice.wait()
# write("C:\\Users\\admin\\Downloads\\bdad-project\\rec-voice.wav", fs, record_voice)
# print("DONE")


recording...
DONE


# Using the model to predict emotion of recorded voice sample

In [15]:
# searching mp3 files in Downloads folder

mp3_files = glob.glob("C:\\Users\\admin\\Downloads\\*.mp3")
print(mp3_files)


['C:\\Users\\admin\\Downloads\\a178cf81-fb6e-4448-919d-a0a91230877e.mp3']


In [19]:
# Converting mp3 files to .wav format

for file in mp3_files:
    # convert wav to mp3
    os.system(f"""ffmpeg -i {file} -acodec pcm_u8 -ar 22050 {file[:-4]}.wav""")   

print('Converted to wave')

Converted to wave


In [20]:
# Extracting features from recorded voice

file = 0
wav_files = glob.glob('C:\\Users\\admin\\Downloads\\*.wav')

for wav_file in wav_files:
    file = wav_file

features = extract_feature(file, mfcc=True, chroma=True, mel=True)


In [22]:
# Predicting the emotion

model.predict([features])

array(['neutral'], dtype='<U9')